In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path as op
import shutil
from zipfile import ZipFile
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from utils import get_image_paths, word_from_image_path, preprocess_image, print_im, TextTransform, N_CHARS, SEQUENCE_LENGTH, IMAGE_DIMENSIONS
from multi_gpu import make_parallel

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


#### Building N-grams Neural Network

In [153]:
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Reshape, Activation, GlobalAveragePooling2D
from keras.layers import Flatten

## Decapitation of previous trained CNN
# cnn = load_model('model_random_weights.keras')
# input = cnn.layers[0].input
# output = cnn.layers[-4].output
# base_ngrams = Model(input=input, output=output)
input_shape = (None, 32, 100, 1,)

ngrams = Sequential()
# ngrams.add(base_ngrams)
ngrams.add(Conv2D(64, 3, 3, batch_input_shape=input_shape, border_mode='same', activation='relu'))
ngrams.add(MaxPooling2D())
ngrams.add(GlobalAveragePooling2D())
ngrams.add(Dense(10000, input_shape=input_shape, activation='sigmoid', name= "10kdense")) 


ngrams.compile(optimizer='adam', loss='categorical_crossentropy')

In [154]:
# ## Parellelize model 
# p_model_ngrams = make_parallel(ngrams, 4)
# p_model_ngrams.compile(optimizer='adam', loss='mse')

### Loading Images

In [155]:
from tqdm import tqdm, tqdm_notebook
base_dir = '/mnt/c/Users/benoua/Google Drive/Master X - Data Sciences/2_Structured Data/Structured-Data/mjsynth'
images_paths = get_image_paths(base_dir)
#%%time
ims = []
errors_1, errors_2 = [], []
for i, im in enumerate(images_paths[:12]):
    try:
        ims.append(preprocess_image(img_to_array(load_img(im, grayscale=True))))
    except OSError:
        errors_1.append(i)

In [156]:
batch = np.stack(ims).reshape(12, 32, 100, 1)
batch.shape

(12, 32, 100, 1)

In [157]:
from utils import NgramTransform

In [158]:
ngramtr = NgramTransform()

In [159]:
batch_y = ngramtr.make_batch_labels(images_paths[:12], False)
batch_y = batch_y.reshape(batch_y.shape[0], batch_y.shape[2]).astype(float)

In [160]:
ngrams.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_5 (Convolution2D)  (None, 32, 100, 64)   640         convolution2d_input_4[0][0]      
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 16, 50, 64)    0           convolution2d_5[0][0]            
____________________________________________________________________________________________________
globalaveragepooling2d_8 (Global (None, 64)            0           maxpooling2d_1[0][0]             
____________________________________________________________________________________________________
10kdense (Dense)                 (None, 10000)         650000      globalaveragepooling2d_8[0][0]   
Total params: 650,640
Trainable params: 650,640
Non-trainable params: 0
___________________

In [163]:
ngrams.fit(batch, batch_y, nb_epoch=10)

Epoch 1/10
12/12 [==============================] - 0s - loss: 126.6781
Epoch 2/10
12/12 [==============================] - 0s - loss: 126.6367
Epoch 3/10
12/12 [==============================] - 0s - loss: 126.6542
Epoch 4/10
12/12 [==============================] - 0s - loss: 126.5998
Epoch 5/10
12/12 [==============================] - 0s - loss: 126.5830
Epoch 6/10
12/12 [==============================] - 0s - loss: 126.5601
Epoch 7/10
12/12 [==============================] - 0s - loss: 126.5304
Epoch 8/10
12/12 [==============================] - 0s - loss: 126.5007
Epoch 9/10
12/12 [==============================] - 0s - loss: 126.4600
Epoch 10/10
12/12 [==============================] - 0s - loss: 126.3826


In [171]:
predictions = ngrams.predict(batch)

In [172]:
predictions = predictions > 0.9

In [173]:
for i in range(12):
#     print(ngramtr.ngram_from_matrix(batch_y[i]))
    print("{} \t\t {}".format(word_from_image_path(images_paths[:12][i]).lower(), ngramtr.ngram_from_matrix(predictions[i])) )

fourteens 		 ['a', 'ac', 'aci', 'acid', 'age', 'an', 'ang', 'ant', 'ante', 'at', 'ate', 'b', 'ba', 'ban', 'bang', 'c', 'ci', 'cid', 'ck', 'cko', 'd', 'di', 'dif', 'difi', 'du', 'dum', 'dumb', 'e', 'eb', 'eba', 'eck', 'ed', 'ee', 'een', 'eens', 'el', 'eli', 'elig', 'en', 'equi', 'es', 'f', 'fi', 'fie', 'fied', 'fou', 'four', 'g', 'ge', 'gh', 'ght', 'ghts', 'gs', 'h', 'he', 'heb', 'ht', 'hts', 'i', 'id', 'idi', 'idif', 'ie', 'ied', 'if', 'ifi', 'ifie', 'ig', 'igh', 'ight', 'il', 'ill', 'illa', 'in', 'ing', 'ipu', 'is', 'isit', 'ist', 'ists', 'it', 'ite', 'iti', 'itin', 'l', 'la', 'lag', 'late', 'li', 'lig', 'ligh', 'llag', 'lt', 'ltr', 'm', 'mb', 'n', 'na', 'nan', 'nant', 'ng', 'ngs', 'ni', 'ning', 'ns', 'nt', 'nte', 'ntes', 'o', 'on', 'oni', 'onin', 'ou', 'our', 'ourt', 'p', 'pi', 'pil', 'pu', 'pul', 'qu', 'qui', 'r', 're', 'rec', 'rel', 'reli', 'req', 'requ', 'rte', 'ru', 'rui', 'ruis', 's', 'sh', 'she', 'si', 'st', 'sti', 'su', 'sui', 'suit', 't', 'te', 'tee', 'tes', 'ti', 'tin', 'tin